# Neural Machine Translation with Attention

Advanced Learning Fall 2024.   
Last updated: 2025-01-12


For SUBMISSION:   

Please upload the complete and executed `ipynb` to your git repository. Verify that all of your output can be viewed directly from github, and provide a link to that git file below.

~~~
STUDENT ID: 208088815
~~~

~~~
STUDENT GIT LINK: MISSING
~~~
In Addition, don't forget to add your ID to the files, and upload to moodle the html version:    
  
`PS3_Attention_2024_ID_[208088815].html`   




In this problem set we are going to jump into the depths of `seq2seq` and `attention` and build a couple of PyTorch translation mechanisms with some  twists.     


*   Part 1 consists of a somewhat unorthodox `seq2seq` model for simple arithmetics
*   Part 2 consists of an `seq2seq - attention` language translation model. We will use it for Hebrew and English.  


---

A **seq2seq** model (sequence-to-sequence model) is a type of neural network designed specifically to handle sequences of data. The model converts input sequences into other sequences of data. This makes them particularly useful for tasks involving language, where the input and output are naturally sequences of words.

Here's a breakdown of how `seq2seq` models work:

* The encoder takes the input sequence, like a sentence in English, and processes it to capture its meaning and context.

* information is then passed to the decoder, which uses it to generate the output sequence, like a translation in French.

* Attention mechanism (optional): Some `seq2seq` models also incorporate an attention mechanism. This allows the decoder to focus on specific parts of the input sequence that are most relevant to generating the next element in the output sequence.

`seq2seq` models are used in many natural language processing (NLP) tasks.



imports: (feel free to add)

In [ ]:
# from __future__ import unicode_literals, print_function, division
# from io import open
# import unicodedata
# Standard library imports
from __future__ import unicode_literals, print_function, division
from io import open
import os
import re
import random
import time
import math
import zipfile
import unicodedata

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# NumPy
import numpy as np

# Matplotlib imports
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')

# Keras imports
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Attention, Flatten, RepeatVector, TimeDistributed, Dense
from keras.callbacks import ModelCheckpoint

# TensorFlow Keras imports
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Dense, Bidirectional, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint

# Device setup for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Part 1: Seq2Seq Arithmetic model

**Using RNN `seq2seq` model to "learn" simple arithmetics!**

> Given the string "54-7", the model should return a prediction: "47".  
> Given the string "10+20", the model should return a prediction: "30".


- Watch Lukas Biewald's short [video](https://youtu.be/MqugtGD605k?si=rAH34ZTJyYDj-XJ1) explaining `seq2seq` models and his toy application (somewhat outdated).
- You can find the code for his example [here](https://github.com/lukas/ml-class/blob/master/videos/seq2seq/train.py).    



1.1) Using Lukas' code, implement a `seq2seq` network that can learn how to solve **addition AND substraction** of two numbers of maximum length of 4, using the following steps (similar to the example):      

* Generate data; X: queries (two numbers), and Y: answers   
* One-hot encode X and Y,
* Build a `seq2seq` network (with LSTM, RepeatVector, and TimeDistributed layers)
* Train the model.
* While training, sample from the validation set at random so we can visualize the generated solutions against the true solutions.    

Notes:  
* The code in the example is quite old and based on Keras. You might have to adapt some of the code to overcome methods/code that is not supported anymore. Hint: for the evaluation part, review the type and format of the "correct" output - this will help you fix the unsupported "model.predict_classes".
* Please use the parameters in the code cell below to train the model.     
* Instead of using a `wandb.config` object, please use a simple dictionary instead.   
* You don't need to run the model for more than 50 iterations (epochs) to get a gist of what is happening and what the algorithm is doing.
* Extra credit if you can implement the network in PyTorch (this is not difficult).    
* Extra credit if you are able to significantly improve the model.

1.2).

a) Do you think this model performs well?  Why or why not?     
b) What are its limitations?   
c) What would you do to improve it?    
d) Can you apply an attention mechanism to this model? Why or why not?   

1.3).  

Add attention to the model. Evaluate the performance against the `seq2seq` you trained above. Which one is performing better?

1.4)

Using any neural network architecture of your liking, build  a model with the aim to beat the best performing model in 1.1 or 1.3. Compare your results in a meaningful way, and add a short explanation to why you think/thought your suggested network is better.

In [ ]:
config = {}
config["training_size"] = 40000
config["digits"] = 4
config["hidden_size"] = 128
config["batch_size"] = 128
config["iterations"] = 50
chars = '0123456789-+ '

SOLUTION:

In [ ]:
### MISSING SOLUTION
# Generationg the data using Lukas' code
# Defining a dictionary to store the model parameters
config_dict = {
    "training_size": 40000,
    "digits": 4,
    "hidden_size": 128,
    "batch_size": 128,
    "iterations" : 50,
    "chars" : '0123456789-+ '
}

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = config_dict['digits'] + 1 + config_dict['digits']

# All the numbers, plus and minus signs and space for padding.
chars = config_dict["chars"]
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')

while len(questions) < config_dict['training_size']:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, config_dict['digits'] + 1))))
    a, b = f(), f()
    # Randomly choose addition or subtraction
    operation = np.random.choice(['+', '-'])

    # Skip any questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = (a, b, operation)
    if key in seen:
        continue
    seen.add(key)

    # Create the question string
    q = f"{a}{operation}{b}"
    query = q + ' ' * (maxlen - len(q))
    # Compute the answer
    ans = str(a + b) if operation == '+' else str(a - b)
    ans += ' ' * (config_dict['digits'] + 1 - len(ans))

    questions.append(query)
    expected.append(ans)

print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), maxlen, len(config_dict['chars'])), dtype=bool)
y = np.zeros((len(questions), config_dict['digits'] + 1, len(config_dict['chars'])), dtype=bool)

for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, maxlen)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, config_dict['digits'] + 1)


# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

Generating data...
Total addition questions: 40000
Vectorization...


In [ ]:
# Building a seq2seq network
# Defining the model
model1 = Sequential()
model1.add(LSTM(config_dict['hidden_size'] , input_shape=(maxlen, len(chars))))
model1.add(RepeatVector(config_dict['digits'] + 1))
model1.add(LSTM(config_dict['hidden_size'], return_sequences=True))
model1.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model1.summary()

# Train the model each generation and show predictions against the validation
# dataset.
checkpoint = ModelCheckpoint('best_model.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)
for iteration in range(config_dict['iterations']):
    print()
    print('-' * 50)
    print('Iteration', iteration)

    model1.fit(x_train, y_train,
              batch_size=config_dict['batch_size'],
              epochs=1,
              validation_data=(x_val, y_val),
              callbacks=[checkpoint])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model1.predict(rowx, verbose=0)
        preds = np.argmax(preds, axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print(guess)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          72,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 5, 128)              │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 5, 13)               │           1,677 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 205,965 (804.55 KB)

 Trainable params: 205,965 (804.55 KB)

...

282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7789 - loss: 0.5977 - val_accuracy: 0.7259 - val_loss: 0.7093
Q 91+9267   T 9358  ☒ 9368 
Q 6579-248  T 6331  ☒ 6436 
Q 44+7      T 51    ☑ 51   
Q 114+1     T 115   ☒ 114  
Q 52-5555   T -5503 ☒ -5597
Q 6+140     T 146   ☑ 146  
Q 3468-2    T 3466  ☒ 3463 
Q 4019-518  T 3501  ☒ 3588 
Q 477-1     T 476   ☒ 477  
Q 1499+8268 T 9767  ☒ 10833

--------------------------------------------------
Iteration 46
281/282 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7849 - loss: 0.5855
Epoch 1: val_loss improved from 0.70935 to 0.70912, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7849 - loss: 0.5855 - val_accuracy: 0.7278 - val_loss: 0.7091
Q 757+87    T 844   ☒ 843  
Q 3393-702  T 2691  ☒ 2577 
Q 7543-82   T 7461  ☒ 7444 
Q 416+17    T 433   ☒ 423  
Q 70+19     T 89    ☒ 88   
Q 717-328   T 389   ☒ 355  
Q 18+733    T 751   ☒ 740  
Q 3283-22   T 3261  ☒ 3263 
Q 5320+7    T 5327  ☒ 5326 
Q 342-14    T 328   ☒ 327  

--------------------------------------------------
Iteration 47
280/282 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7884 - loss: 0.5771
Epoch 1: val_loss improved from 0.70912 to 0.70533, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7884 - loss: 0.5772 - val_accuracy: 0.7306 - val_loss: 0.7053
Q 29-4710   T -4681 ☒ -4672
Q 1337-67   T 1270  ☒ 1392 
Q 11-4998   T -4987 ☒ -4992
Q 99-2808   T -2709 ☒ -2790
Q 16+536    T 552   ☑ 552  
Q 506+34    T 540   ☒ 551  
Q 468-228   T 240   ☒ 226  
Q 721-16    T 705   ☑ 705  
Q 886-554   T 332   ☒ 323  
Q 3793-893  T 2900  ☒ 2928 

--------------------------------------------------
Iteration 48
280/282 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7919 - loss: 0.5672
Epoch 1: val_loss improved from 0.70533 to 0.70393, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7919 - loss: 0.5672 - val_accuracy: 0.7316 - val_loss: 0.7039
Q 37-98     T -61   ☒ -51  
Q 423-75    T 348   ☒ 356  
Q 101+60    T 161   ☒ 162  
Q 6367+256  T 6623  ☒ 6712 
Q 34-314    T -280  ☒ -200 
Q 728-95    T 633   ☒ 646  
Q 9+1230    T 1239  ☒ 1231 
Q 8-662     T -654  ☑ -654 
Q 3-5359    T -5356 ☒ -5355
Q 5771+596  T 6367  ☒ 6464 

--------------------------------------------------
Iteration 49
279/282 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7923 - loss: 0.5620
Epoch 1: val_loss improved from 0.70393 to 0.69383, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7923 - loss: 0.5620 - val_accuracy: 0.7317 - val_loss: 0.6938
Q 130-9952  T -9822 ☒ -9828
Q 691+98    T 789   ☒ 781  
Q 5796-52   T 5744  ☒ 5736 
Q 8864-8    T 8856  ☒ 8848 
Q 9898+9483 T 19381 ☒ 18123
Q 7259-6344 T 915   ☒ 11   
Q 976-4     T 972   ☑ 972  
Q 298+6498  T 6796  ☒ 6877 
Q 918+9775  T 10693 ☒ 10650
Q 9-5801    T -5792 ☑ -5792


a) I think the model performs reasonably well, with a training accuracy of about 80% and validation accuracy of about 73%. These results suggest that the model is able to learn and generalize on the task to some extent. It’s possible that the model is not fully capturing the necessary dependencies between the inputs and outputs, as reflected by some of the errors in the output predictions. In the examples from the validation data, it seems that surprisingly, the model most of the time works in estimating the answer.

b)
One limitation of this model is its inability to effectively capture long-range dependencies or focus on the most relevant parts of the input sequence for each prediction. Without an attention mechanism, the model struggles to selectively prioritize different parts of the input, which is crucial in tasks where the input sequence may vary significantly and require different areas of focus for each prediction(like this task). This lack of flexibility can hinder performance, especially when dealing with complex or nuanced patterns. Additionally, the simple encoder-decoder structure limits the model’s ability to handle more intricate dependencies, reducing its generalization to more complex inputs.

c)  Improvements:
To improve this model, I would suggest implementing an attention mechanism. Attention would allow the model to focus on different parts of the sequence, which is especially useful in sequence-to-sequence tasks where important information may be scattered throughout the input.
Furthermore, hyperparameter tuning could help, such as adjusting the learning rate, increasing the number of layers, or adding more neurons per layer. Regularization techniques like dropout could help prevent overfitting and improve generalization.
Additionally, expanding the training data could help the model generalize better to a wider range of inputs. We can balance the dataset between operations to ensure the model works well for sunstraction and addition.

d) Yes, an attention mechanism can be applied to this model, and it would be particularly beneficial for the current problem of solving arithmetic equations. The simple encoder-decoder structure without attention faces challenges in handling long input sequences or varying complexity in equations.

In this problem, an attention mechanism would allow the model to focus on relevant parts of the input sequence (such as specific digits or operators) when making predictions, rather than relying on a fixed-length representation of the entire sequence. This would enable the model to handle longer sequences more effectively, improve its generalization to larger numbers, and reduce errors in operations.

## 1.3


In [ ]:
# Defining model 2 - adding attention
# Defining input layer
inputs = Input(shape=(maxlen, len(chars)))

# First LSTM layer - Output sequences to feed into attention
lstm_1 = LSTM(config_dict['hidden_size'], return_sequences=True)(inputs)

# Attention layer - Attention mechanism applied on lstm_1 output
attention = Attention()([lstm_1, lstm_1])

# Flattenning the attention output to match the expected input shape for RepeatVector
attention_flattened = Flatten()(attention)

# Repeat Vector layer to ensure that the output has the same shape as the original sequence length
repeat_vector = RepeatVector(config_dict['digits'] + 1)(attention_flattened)

# Second LSTM layer - Output sequences
lstm_2 = LSTM(config_dict['hidden_size'], return_sequences=True)(repeat_vector)

# TimeDistributed Dense layer for output (categorical distribution)
outputs = TimeDistributed(Dense(len(chars), activation='softmax'))(lstm_2)

# Creating the model
model2 = Model(inputs=inputs, outputs=outputs)

# Compile the model
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model2.summary()

# Train the model each generation and show predictions against the validation
# dataset.
checkpoint = ModelCheckpoint('best_model.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)
for iteration in range(config_dict['iterations']):
    print()
    print('-' * 50)
    print('Iteration', iteration)

    model2.fit(x_train, y_train,
              batch_size=config_dict['batch_size'],
              epochs=1,
              validation_data=(x_val, y_val),
              callbacks=[checkpoint])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model2.predict(rowx, verbose=0)
        preds = np.argmax(preds, axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print(guess)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 9, 13)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ (None, 9, 128)         │         72,704 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_1 (Attention)   │ (None, 9, 128)         │              0 │ lstm_6[0][0],          │
│                           │                        │                │ lstm_6[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 1152)           │              0 │ attention_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_vector_3           │ (None, 5, 1152)        │              0 │ flatten_1[0][0]        │
│ (RepeatVector)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ (None, 5, 128)         │        655,872 │ repeat_vector_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_3        │ (None, 5, 13)          │          1,677 │ lstm_7[0][0]           │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 730,253 (2.79 MB)

 Trainable params: 730,253 (2.79 MB)

 Non-trainable params: 0 (0.00 B)

...

282/282 ━━━━━━━━━━━━━━━━━━━━ 34s 121ms/step - accuracy: 0.9025 - loss: 0.2941 - val_accuracy: 0.8251 - val_loss: 0.4936
Q 55+9      T 64    ☑ 64   
Q 2+536     T 538   ☑ 538  
Q 873+8     T 881   ☒ 891  
Q 978+25    T 1003  ☒ 103  
Q 0-4       T -4    ☑ -4   
Q 2675+143  T 2818  ☒ 2778 
Q 7047+9364 T 16411 ☒ 16801
Q 860+25    T 885   ☑ 885  
Q 4-7051    T -7047 ☑ -7047
Q 1171-8122 T -6951 ☒ -7039

--------------------------------------------------
Iteration 43
281/282 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9085 - loss: 0.2801
Epoch 1: val_loss did not improve from 0.49363
282/282 ━━━━━━━━━━━━━━━━━━━━ 34s 119ms/step - accuracy: 0.9085 - loss: 0.2801 - val_accuracy: 0.8257 - val_loss: 0.5036
Q 484+675   T 1159  ☒ 1129 
Q 42+9628   T 9670  ☒ 9680 
Q 7-1073    T -1066 ☑ -1066
Q 71-278    T -207  ☑ -207 
Q 7+5297    T 5304  ☒ 5294 
Q 2663-13   T 2650  ☒ 2640 
Q 723+199   T 922   ☒ 802  
Q 25-119    T -94   ☑ -94  
Q 25-4      T 21    ☑ 21   
Q 858+19    T 877   ☑ 877  

----------

282/282 ━━━━━━━━━━━━━━━━━━━━ 38s 133ms/step - accuracy: 0.9187 - loss: 0.2544 - val_accuracy: 0.8298 - val_loss: 0.4892
Q 8017+36   T 8053  ☒ 8001 
Q 8-1896    T -1888 ☑ -1888
Q 3879+3    T 3882  ☒ 3890 
Q 373+42    T 415   ☒ 405  
Q 39-1412   T -1373 ☒ -1473
Q 845+4175  T 5020  ☒ 4900 
Q 1848+274  T 2122  ☒ 2012 
Q 127-8831  T -8704 ☒ -8604
Q 928-6     T 922   ☑ 922  
Q 1702+843  T 2545  ☑ 2545 

--------------------------------------------------
Iteration 46
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9199 - loss: 0.2478
Epoch 1: val_loss improved from 0.48923 to 0.48739, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 35s 124ms/step - accuracy: 0.9199 - loss: 0.2479 - val_accuracy: 0.8322 - val_loss: 0.4874
Q 6+656     T 662   ☑ 662  
Q 93-87     T 6     ☑ 6    
Q 8+451     T 459   ☑ 459  
Q 3430+4190 T 7620  ☒ 8960 
Q 9081-5    T 9076  ☒ 9086 
Q 780-632   T 148   ☒ 142  
Q 2-45      T -43   ☑ -43  
Q 5136+12   T 5148  ☑ 5148 
Q 82-7575   T -7493 ☒ -7463
Q 22+135    T 157   ☑ 157  

--------------------------------------------------
Iteration 47
281/282 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9241 - loss: 0.2394
Epoch 1: val_loss improved from 0.48739 to 0.48689, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 36s 128ms/step - accuracy: 0.9241 - loss: 0.2394 - val_accuracy: 0.8309 - val_loss: 0.4869
Q 1-814     T -813  ☑ -813 
Q 27+26     T 53    ☒ 43   
Q 75+6982   T 7057  ☒ 7049 
Q 4+701     T 705   ☑ 705  
Q 221-35    T 186   ☑ 186  
Q 50-7117   T -7067 ☒ -7077
Q 51+7      T 58    ☑ 58   
Q 4799+5    T 4804  ☒ 4794 
Q 333+5579  T 5912  ☒ 5902 
Q 6-9977    T -9971 ☒ -9981

--------------------------------------------------
Iteration 48
281/282 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.9275 - loss: 0.2291
Epoch 1: val_loss improved from 0.48689 to 0.48322, saving model to best_model.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 35s 122ms/step - accuracy: 0.9275 - loss: 0.2291 - val_accuracy: 0.8361 - val_loss: 0.4832
Q 3791-6804 T -3013 ☒ -4063
Q 53+4      T 57    ☑ 57   
Q 42+4509   T 4551  ☒ 4531 
Q 42-2      T 40    ☑ 40   
Q 978-91    T 887   ☑ 887  
Q 7390-258  T 7132  ☑ 7132 
Q 36+60     T 96    ☑ 96   
Q 33+2      T 35    ☑ 35   
Q 807-569   T 238   ☑ 238  
Q 6+957     T 963   ☑ 963  

--------------------------------------------------
Iteration 49
281/282 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9294 - loss: 0.2228
Epoch 1: val_loss did not improve from 0.48322
282/282 ━━━━━━━━━━━━━━━━━━━━ 36s 128ms/step - accuracy: 0.9294 - loss: 0.2228 - val_accuracy: 0.8334 - val_loss: 0.4854
Q 159+9042  T 9201  ☒ 9631 
Q 9474+2468 T 11942 ☒ 12622
Q 897-726   T 171   ☒ 211  
Q 539+26    T 565   ☒ 573  
Q 1176-7    T 1169  ☑ 1169 
Q 17+86     T 103   ☑ 103  
Q 2031+7042 T 9073  ☒ 9023 
Q 7391-93   T 7298  ☒ 7288 
Q 85-4      T 81    ☑ 81   
Q 8318+27   T 8345  ☒ 8245 


It can be seen that in the initial model (without attention), the accuracy was almost 80%, with a validation accuracy of nearly 73%. The model struggled with handling large numbers and had an inconsistent performance on basic arithmetic problems.

After adding the attention mechanism, the accuracy jumped to 92%, with a validation accuracy of 83%, and the loss decreased. These results indicate a significant improvement in performance, especially in terms of both accuracy and loss. The attention mechanism helped the model focus on the most relevant parts of the input sequence, enabling it to handle more complex arithmetic operations and improve its generalization capabilities.

In comparison, the attention-based model performed better across both training and validation sets, suggesting that the attention mechanism greatly enhanced the model's ability to solve arithmetic problems accurately.

## 1.4

In [ ]:
# Defining the Model 3 architecture
def build_model_3(input_shape, output_shape, n_chars):
    inputs = Input(shape=input_shape)

    # Bidirectional LSTM - Capture both forward and backward context
    lstm_1 = Bidirectional(LSTM(config_dict['hidden_size'], return_sequences=True))(inputs)

    # Dropout layer to prevent overfitting
    lstm_1 = Dropout(0.2)(lstm_1)

    # Attention mechanism
    attention = Attention()([lstm_1, lstm_1])

    # Flatten the attention output and repeat to match the decoder input shape
    attention_flattened = Flatten()(attention)
    repeat_vector = RepeatVector(output_shape[0])(attention_flattened)

    # Second LSTM layer
    lstm_2 = Bidirectional(LSTM(config_dict['hidden_size'], return_sequences=True))(repeat_vector)

    # Dropout layer for regularization
    lstm_2 = Dropout(0.2)(lstm_2)

    # TimeDistributed Dense layer to generate the final predictions
    outputs = TimeDistributed(Dense(n_chars, activation='softmax'))(lstm_2)

    model_3 = Model(inputs=inputs, outputs=outputs)
    model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model_3

# Input shape: (maxlen, len(chars)), Output shape: (digits + 1, len(chars))
input_shape = (x_train.shape[1], x_train.shape[2])
output_shape = (y_train.shape[1], y_train.shape[2])

# Build and summarize the model
model_3 = build_model_3(input_shape, output_shape, len(config_dict['chars']))
model_3.summary()

# Training the model and saving the best model
checkpoint = ModelCheckpoint('best_model_3.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)

for iteration in range(config_dict['iterations']):
    print()
    print('-' * 50)
    print('Iteration', iteration)

    model_3.fit(x_train, y_train,
                batch_size=config_dict['batch_size'],
                epochs=1,
                validation_data=(x_val, y_val),
                callbacks=[checkpoint])

    # Select 10 samples from the validation set at random for error visualization
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]

        preds = model_3.predict(rowx, verbose=0)
        preds = np.argmax(preds, axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)

        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print(guess)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 9, 13)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 9, 256)         │        145,408 │ input_layer_1[0][0]    │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 9, 256)         │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)     │ (None, 9, 256)         │              0 │ dropout[0][0],         │
│                           │                        │                │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 2304)           │              0 │ attention[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_vector_1           │ (None, 5, 2304)        │              0 │ flatten[0][0]          │
│ (RepeatVector)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 5, 256)         │      2,491,392 │ repeat_vector_1[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 5, 256)         │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_1        │ (None, 5, 13)          │          3,341 │ dropout_1[0][0]        │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,640,141 (10.07 MB)

 Trainable params: 2,640,141 (10.07 MB)

 Non-trainable params: 0 (0.00 B)

...

282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9524 - loss: 0.1280 - val_accuracy: 0.9287 - val_loss: 0.1963
Q 909-10    T 899   ☑ 899  
Q 3576+9    T 3585  ☑ 3585 
Q 747-1820  T -1073 ☒ -1173
Q 48+79     T 127   ☑ 127  
Q 2+2300    T 2302  ☑ 2302 
Q 785+1     T 786   ☑ 786  
Q 1+3568    T 3569  ☑ 3569 
Q 150+0     T 150   ☑ 150  
Q 70-7      T 63    ☑ 63   
Q 433+938   T 1371  ☑ 1371 

--------------------------------------------------
Iteration 35
280/282 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9546 - loss: 0.1239
Epoch 1: val_loss improved from 0.19631 to 0.19630, saving model to best_model_3.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9546 - loss: 0.1239 - val_accuracy: 0.9295 - val_loss: 0.1963
Q 3-2025    T -2022 ☑ -2022
Q 49-31     T 18    ☑ 18   
Q 46-33     T 13    ☑ 13   
Q 1535+4105 T 5640  ☒ 5650 
Q 2353-8270 T -5917 ☒ -6937
Q 3688-666  T 3022  ☒ 2922 
Q 6376-6    T 6370  ☑ 6370 
Q 2-768     T -766  ☑ -766 
Q 4890+1    T 4891  ☑ 4891 
Q 533-603   T -70   ☒ -40  

--------------------------------------------------
Iteration 36
278/282 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9562 - loss: 0.1184
Epoch 1: val_loss improved from 0.19630 to 0.19327, saving model to best_model_3.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9562 - loss: 0.1184 - val_accuracy: 0.9309 - val_loss: 0.1933
Q 7-324     T -317  ☑ -317 
Q 5-63      T -58   ☑ -58  
Q 366+8     T 374   ☑ 374  
Q 5+3796    T 3801  ☑ 3801 
Q 3774+8    T 3782  ☑ 3782 
Q 867+7281  T 8148  ☒ 8158 
Q 51+291    T 342   ☑ 342  
Q 671-759   T -88   ☒ -98  
Q 237+4     T 241   ☑ 241  
Q 1-1777    T -1776 ☑ -1776

--------------------------------------------------
Iteration 37
279/282 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9556 - loss: 0.1186
Epoch 1: val_loss improved from 0.19327 to 0.18916, saving model to best_model_3.h5


282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9556 - loss: 0.1186 - val_accuracy: 0.9304 - val_loss: 0.1892
Q 645+497   T 1142  ☑ 1142 
Q 9650+1495 T 11145 ☒ 10155
Q 1124-345  T 779   ☑ 779  
Q 37-801    T -764  ☑ -764 
Q 306-1     T 305   ☑ 305  
Q 1248-57   T 1191  ☑ 1191 
Q 0+7057    T 7057  ☑ 7057 
Q 3+246     T 249   ☑ 249  
Q 7434-63   T 7371  ☑ 7371 
Q 18+86     T 104   ☑ 104  

--------------------------------------------------
Iteration 38
279/282 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9608 - loss: 0.1072
Epoch 1: val_loss did not improve from 0.18916
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9608 - loss: 0.1072 - val_accuracy: 0.9312 - val_loss: 0.1918
Q 4-983     T -979  ☒ -989 
Q 368-862   T -494  ☒ -504 
Q 12+9766   T 9778  ☑ 9778 
Q 476+77    T 553   ☑ 553  
Q 39+6126   T 6165  ☑ 6165 
Q 248+32    T 280   ☒ 270  
Q 1594+95   T 1689  ☒ 1699 
Q 99-91     T 8     ☒ 0    
Q 134+7     T 141   ☑ 141  
Q 38+6      T 44    ☑ 44   

---------------

282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9625 - loss: 0.1030 - val_accuracy: 0.9357 - val_loss: 0.1836
Q 3233-4    T 3229  ☑ 3229 
Q 198-3018  T -2820 ☒ -2920
Q 1-3451    T -3450 ☑ -3450
Q 5+875     T 880   ☑ 880  
Q 0+37      T 37    ☑ 37   
Q 5+550     T 555   ☑ 555  
Q 829-1674  T -845  ☑ -845 
Q 387+7     T 394   ☑ 394  
Q 0-10      T -10   ☑ -10  
Q 359-2     T 357   ☑ 357  

--------------------------------------------------
Iteration 43
281/282 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9658 - loss: 0.0924
Epoch 1: val_loss did not improve from 0.18361
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9658 - loss: 0.0924 - val_accuracy: 0.9345 - val_loss: 0.1933
Q 940+5330  T 6270  ☒ 6370 
Q 232-0     T 232   ☑ 232  
Q 666+764   T 1430  ☒ 1420 
Q 768-3     T 765   ☑ 765  
Q 2-3321    T -3319 ☑ -3319
Q 3875-143  T 3732  ☒ 3722 
Q 3-4040    T -4037 ☑ -4037
Q 716-9940  T -9224 ☒ -9164
Q 5929+527  T 6456  ☒ 6466 
Q 849+246   T 1095  ☒ 1055 

---------------

The Improved Model (Model 3) shows significant improvements over model 2 including the attention, in terms of both training and validation performance. The training accuracy of Model 3 is about 97%, compared to the second model’s 93%.The validation accuracy of Model 3 is almost 94%, a 11% improvement over the second model’s 83%. This demonstrates that Model 3 generalizes much better to unseen data, likely due to the enhancements made in the architecture.

I made two main changes to improve the model. First, I used bidirectional LSTMs so the network is able to capture context from both past and future tokens in the input sequence. This allows the model to have a richer understanding of the sequence, which helps improve predictions.

Second, I add dropout to the model prevents it from overfitting to the training data. It forces the model to learn more robust features that generalize better to unseen data. This is reflected in the significant improvement in validation accuracy for Model 3.